<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/ByNumWord_Cluster_Character_based_clustering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re

directory = '/content/drive/MyDrive'
df_base = pd.read_csv(os.path.join(directory, 'All_titles.csv'))
df_base.rename(columns={'0': 'title'}, inplace=True)
df_base.drop(columns=['Unnamed: 0'], inplace=True)
df = df_base.sample(n=10000).dropna()


Mounted at /content/drive


In [3]:
df.head()

,title
1737288,Towards inclusive cities and social sustainabi...
3220253,The impact of increased interconnection on ele...
4004390,Increasing older persons' employment in Finlan...
5530736,Cutting the cost of computing
10241542,Executive Pay in the Public Sector: The Case o...


In [2]:
from string import printable
from collections import Counter
import numpy as np

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    """ Returns a vector with counts for each printable character found in a title """
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

In [3]:
vects = []
for title in df.title:
    vects.append(vectorize(title))

In [5]:
title

'Corporate responsibility for economic, social and cultural rights: Rights in search of a remedy?'

In [ ]:
vects
#vectorize('Corporate responsibility for economic, social and cultural rights: Rights in search of a remedy?')

In [7]:
vectorize('Corporate responsibility for economic, social and cultural rights: Rights in search of a remedy?')

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  1,  5,  2,  6,  2,  2,
        3,  8,  0,  0,  4,  2,  4,  8,  2,  0,  8,  6,  5,  2,  0,  0,  0,
        2,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,  0])

In [ ]:
%%time
threshold = 10
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df.iloc[i].title
                        t2 = df.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)

In [ ]:
# when sample = 2k
#%%time
#from sklearn.neighbors import KDTree
#tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')

CPU times: user 551 ms, sys: 136 ms, total: 687 ms
Wall time: 1.29 s


In [ ]:
%%time
from sklearn.neighbors import KDTree
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')

CPU times: user 116 ms, sys: 922 µs, total: 117 ms
Wall time: 117 ms


In [ ]:
for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=3)
    print(dist)

In [ ]:

duplicate_rows = df[df.duplicated(subset='title')]
print(duplicate_rows)


                                                      title
1053159                                      At Close Range
471596                                            Editorial
10601686                                          Editorial
5834      Persuasive Advertising and Product Differentia...
8331533                                           Editorial
119442           [Notifiable Diseases and Mortality Tables]
10745515                   Uncertainty and Growth Disasters
2561043                                      Books Received
2028786   Price and quantity controls as facilitating de...
8122187                                        Book Reviews
811896                                       At Close Range


In [ ]:
len(duplicate_rows)/len(df)

0.0011038635223281485

# with less threshold

In [8]:
%%time
threshold = 5
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df.iloc[i].title
                        t2 = df.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)

Official Publications Received
Official Publications Received
____________________________________________________________________________________________________
US quantitative easing and firm’s default risk: The role of Corporate Social Responsibility (CSR)
US quantitative easing and firm’s default risk: The role of Corporate Social Responsibility (CSR)
____________________________________________________________________________________________________
US quantitative easing and firm’s default risk: The role of Corporate Social Responsibility (CSR)
US quantitative easing and firm’s default risk: The role of Corporate Social Responsibility (CSR)
____________________________________________________________________________________________________
Official Publications Received
Official Publications Received
____________________________________________________________________________________________________
CPU times: user 38min 13s, sys: 1min 49s, total: 40min 3s
Wall time: 39min 27s


In [9]:
%%time
from sklearn.neighbors import KDTree
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')

CPU times: user 1.13 s, sys: 294 ms, total: 1.42 s
Wall time: 1.97 s


In [ ]:
for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=3)
    print(dist)

In [11]:
duplicate_rows = df[df.duplicated(subset='title')]
print(duplicate_rows)


                                                     title
875959                                           Editorial
8792239                                         Discussion
6301072  US quantitative easing and firm’s default risk...
2434640                                  England and Wales
60295                                     Political Review
1015763                                     At Close Range
4199185                     Official Publications Received


In [12]:
len(duplicate_rows)/len(df)

0.0007020359041219537

# From here

#run it with threshold = 10

In [6]:
from string import printable
from collections import Counter
import numpy as np

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    """ Returns a vector with counts for each printable character found in a title """
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df.title:
    vects.append(vectorize(title))

In [7]:
%%time
threshold = 10
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df.iloc[i].title
                        t2 = df.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)


Economic History and Cliometrics
Political and Economic History
____________________________________________________________________________________________________
List of Abbreviations
Abbreviations and notes
____________________________________________________________________________________________________
List of Abbreviations
List of Abbreviations
____________________________________________________________________________________________________
Panel Data Econometrics
Social Sector Management
____________________________________________________________________________________________________
Political and Economic History
Economic History and Cliometrics
____________________________________________________________________________________________________
The Free Market and Morality
Foreign Trade and the Money Market
____________________________________________________________________________________________________
Демографические процессы в Омской области в 1990-2004 гг. И их 

In [8]:
%%time
from sklearn.neighbors import KDTree
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')


CPU times: user 996 ms, sys: 305 ms, total: 1.3 s
Wall time: 2.09 s


In [9]:

print("===========================")
for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=3)

print("===========================")
duplicate_rows = df[df.duplicated(subset='title')]
print(duplicate_rows)

print("===========================")
len(duplicate_rows)/len(df)

                                                      title
940876    TABLE I. CASES OF SPECIFIED NOTIFIABLE DISEASE...
6369058                               List of Abbreviations
5273028                                           EDITORIAL
10100962                                          Editorial
6359687                                           Editorial
803443                                            Editorial
1157525   The Impact of Expectations, Match Importance, ...
4830403                                        Book Reviews
8353434                                           Editorial
5234831                                        Book Reviews
4256929                                   England and Wales
9597596                                           Editorial
116553           [Notifiable Diseases and Mortality Tables]
152485                             Current Labor Statistics
94907                                           Back Matter
312628                                  

0.001905717151454363

# gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'k': [1, 3, 5, 7],  # Different values for the number of neighbors (K)
    'threshold': np.arange(5, 15, 2)  # Range of threshold values
}

In [ ]:
def evaluate_knn(vects, k, threshold, tree):
  total_similar = 0
  for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=k)
    similar_titles = np.where(dist <= threshold)[0]  # Titles within threshold
    total_similar += len(similar_titles) - 1  # Exclude itself
  return total_similar

In [ ]:
from sklearn.neighbors import KDTree

tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')
knn_model = KDTreeNeighbors(tree)

grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, scoring=evaluate_knn, cv=5)
grid_search.fit(vects, df)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")